### Tools

In [1]:
#%pip install -U langchain-community langgraph langchain-openai tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.3 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 8.7 MB/s eta 0:00:00ta 0:00:01
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.21
    Uninstalling langchain-core-0.2.21:
      Successfully uninstalled langchain-core-0.2.21
  Attempting uninstall: langgraph
    Found existing installation: langgraph 0.1.8
    Uninstalling langgraph-0.1.8:
      Successfully uninstalled langgraph-0.1.8
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.2.7
    Uninstalling langchain-community-0.2.7:
      Successfully uninstalled langchain-community-0.2.7
Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os

# os.environ["LANGCHAIN_TRACING_V2"] = "true"
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

In [3]:
from langchain_community.tools.tavily_search import TavilySearchResults

# define search tool

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Los Angeles")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

/Users/rickchakra/Projects/aiug_workshop/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Los Angeles', 'region': 'California', 'country': 'United States of America', 'lat': 34.05, 'lon': -118.24, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1722430136, 'localtime': '2024-07-31 5:48'}, 'current': {'last_updated_epoch': 1722429900, 'last_updated': '2024-07-31 05:45', 'temp_c': 20.1, 'temp_f': 68.3, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 193, 'wind_dir': 'SSW', 'pressure_mb': 1012.0, 'pressure_in': 29.89, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 59, 'cloud': 0, 'feelslike_c': 20.1, 'feelslike_f': 68.3, 'windchill_c': 20.1, 'windchill_f': 68.3, 'heatindex_c': 20.2, 'heatindex_f': 68.4, 'dewpoint_c': 11.8, 'dewpoint_f': 53.2, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 1.0, 'gust_mph': 3.0, 'gust_kph': 4.8}}"}, {'url': 'https://forecast.weather.gov/zipcity.php?inputstr

### Bind model with tools

In [5]:
#pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

# initialize model
model = ChatOpenAI(model="gpt-4o")

In [5]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [6]:
# bind tools with model
model_with_tools = model.bind_tools(tools)

In [7]:
# check response of model with search tool (no tool call)
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []


In [15]:
# check response of model with search tool (tool call)
response = model_with_tools.invoke([HumanMessage(content="What's the weather in San Francisco?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'call_vsqUjLdYi76BbggN09chUFAp', 'type': 'tool_call'}]


### Using a prebuilt react agent with tools

In [12]:
from langgraph.prebuilt import create_react_agent

#use prebuilt ReAct agent
agent_executor = create_react_agent(model, tools)

In [13]:
# test agent response (no tool call)
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='373647a3-9341-4109-aafa-2325929bb133'),
 AIMessage(content='Hello! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 81, 'total_tokens': 91}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'stop', 'logprobs': None}, id='run-360d85a7-8803-4c6f-a87b-38fdff89fa33-0', usage_metadata={'input_tokens': 81, 'output_tokens': 10, 'total_tokens': 91})]

In [12]:
# test agent response (tool call)
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the weather in San Francisco?")]}
)
response["messages"]

[HumanMessage(content="What's the weather in Los Angeles?", id='1ef3608b-a69e-47ed-9ff1-c09f188fe0ee'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_p60p35lVmmyE4qn6JbZsZbaI', 'function': {'arguments': '{"query":"current weather in Los Angeles"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 86, 'total_tokens': 108}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_400f27fa1f', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e2604cff-b551-4e91-bb42-5bd27f2b665b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'call_p60p35lVmmyE4qn6JbZsZbaI', 'type': 'tool_call'}], usage_metadata={'input_tokens': 86, 'output_tokens': 22, 'total_tokens': 108}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\'